In [ ]:
pip install transformers torch


     |████████████████████████████████| 1.3MB 10.5MB/s 
     |████████████████████████████████| 1.1MB 44.3MB/s 
     |████████████████████████████████| 2.9MB 39.5MB/s 
     |████████████████████████████████| 890kB 53.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5b809b0651bd8985694a9c4846ce8083f77f2d582fb8886ea8022a0e7731bb77
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
torch.cuda.is_available()

False

In [ ]:
# model = T5ForConditionalGeneration.from_pretrained("t5-base").to('cuda:0')
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")


# model = T5ForConditionalGeneration.from_pretrained("t5-small").to('cuda:0')
# model = T5ForConditionalGeneration.from_pretrained("t5-small")
# tokenizer = T5Tokenizer.from_pretrained("t5-small")



In [ ]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

path_test= '/content/drive/My Drive/Colab Notebooks/data/test_dataset.pkl'

stories = pickle.load(open(path_test, 'rb'))
print("total number of CNN data: ",len(stories))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total number of CNN data:  10000


In [ ]:
import pandas as pd
df = pd.DataFrame(stories)

def shorten_sen(list_sen, NUM_SEN=2):
  final_sen =[]
  if len(list_sen) < NUM_SEN:
    NUM_SEN = len(list_sen)
  for i in range(NUM_SEN):
    final_sen.append(list_sen[i])
  return '. '.join(final_sen)# + '.'

df['shorten_story'] = df.apply(lambda row: shorten_sen(row['story']), axis=1)
df['shorten_highlights'] = df.apply(lambda row: row['highlights'][0], axis=1)

df['predicted_highlights'] = '-'
df

# df.iloc[0,4]#, df.iloc[0,0] 

,story,highlights,id,shorten_story,shorten_highlights,predicted_highlights
0,[ever hopped in a taxi and sat in traffic for ...,[mellowcab is a threewheeled vehicle that the ...,id_cnn_88698,ever hopped in a taxi and sat in traffic for h...,mellowcab is a threewheeled vehicle that the o...,-
1,[a mother of six has claimed she was threatene...,[amanda moss and husband simon booked family h...,id_dm_26727,a mother of six has claimed she was threatened...,amanda moss and husband simon booked family ho...,-
2,[thomas morris who was strangled by his own ka...,[thomas morris was found dead in home gym in m...,id_dm_205431,thomas morris who was strangled by his own kar...,thomas morris was found dead in home gym in mo...,-
3,[britains princess anne and major winner annik...,"[ra announces first women members, princess an...",id_cnn_47838,britains princess anne and major winner annika...,ra announces first women members,-
4,[it is alleged that michael karkoc was an offi...,[alleged michael karkoc was in the ssled ukran...,id_dm_63684,it is alleged that michael karkoc was an offic...,alleged michael karkoc was in the ssled ukrani...,-
...,...,...,...,...,...,...
9995,[an indonesian lawyer for condemned bali nine ...,[lawyer for bali nine duo makes plea on social...,id_dm_183675,an indonesian lawyer for condemned bali nine d...,lawyer for bali nine duo makes plea on social ...,-
9996,[campaigners stamping antiisrael propalestine ...,[british banknotes marked with message free pa...,id_dm_14557,campaigners stamping antiisrael propalestine m...,british banknotes marked with message free pal...,-
9997,[the solution to ending an abusive relationshi...,[authors say leaving abuse can be a confusing ...,id_cnn_90018,the solution to ending an abusive relationship...,authors say leaving abuse can be a confusing p...,-
9998,[in compensation for a small piece of land tak...,[victorian government pays melbournes scotch c...,id_dm_192809,in compensation for a small piece of land take...,victorian government pays melbournes scotch co...,-


In [ ]:
%%time
from tqdm import tqdm

# for i in tqdm(range(0,10)):
for i in tqdm(range(0,len(df_test))):

    article= df.loc[i,'shorten_story']
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("summarize: " + article,
                          return_tensors="pt", max_length=200, truncation=True)#.to('cuda:0')
                          # return_tensors="pt", max_length=512, truncation=True).to('cuda:0')

    # generate the summarization output
    outputs = model.generate(
        inputs,
        max_length=30,
        min_length=10,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True)
    df.loc[i,'predicted_highlights']=tokenizer.decode(outputs[0])
    print()

 10%|█         | 1/10 [00:04<00:44,  4.90s/it]

 20%|██        | 2/10 [00:08<00:36,  4.61s/it]

 30%|███       | 3/10 [00:12<00:30,  4.36s/it]

 40%|████      | 4/10 [00:16<00:25,  4.23s/it]

 50%|█████     | 5/10 [00:20<00:20,  4.16s/it]

...

 70%|███████   | 7/10 [00:28<00:11,  3.98s/it]

 80%|████████  | 8/10 [00:32<00:08,  4.05s/it]

 90%|█████████ | 9/10 [00:35<00:03,  3.85s/it]

100%|██████████| 10/10 [00:39<00:00,  3.98s/it]


CPU times: user 38.7 s, sys: 965 ms, total: 39.7 s
Wall time: 39.8 s


In [ ]:
loc_save = '/content/drive/My Drive/Colab Notebooks/data'

df.to_csv(loc_save+'/saved_predicted_sum.csv')

In [ ]:
index = 1
df.iloc[index,4], df.iloc[index,0],df.iloc[index,1], df.iloc[index,3] 

('family claims jerald newman had video game down his pants while protecting child',
 ['the full story behind the shoplifting grandfather body slammed to the ground emerged today as the nation searched its soul following the chaotic sometimes violent scenes of greed on black friday',
  'jerald allen newman has come to symbolise the mayhem of the day after he was videoed with blood streaming down his face as he was arrested by police officers in phoenix arizona',
  'the police say he was shoplifting and sustained the injuries because he resisted arrest and became aggressive after being found with a video game down his pants',
  'but mr newmans tearful eightyearold grandson nick',
  'has spoken out to defend his grandfather to abc news saying i only got one game and',
  'people were trying to take it away from me and put it under his shirt so',
  'no one would take it',
  'scroll down for video',
  'out cold jerald allen newman was knocked unconscious after a police officer kicked his le

In [ ]:
import rouge


def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg', 'Best', 'Individual']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    # hypothesis_1 = "King Norodom Sihanouk has declined requests to chair a summit of Cambodia 's top political leaders , saying the meeting would not bring any progress in deadlocked negotiations to form a government .\nGovernment and opposition parties have asked King Norodom Sihanouk to host a summit meeting after a series of post-election negotiations between the two opposition groups and Hun Sen 's party to form a new government failed .\nHun Sen 's ruling party narrowly won a majority in elections in July , but the opposition _ claiming widespread intimidation and fraud _ has denied Hun Sen the two-thirds vote in parliament required to approve the next government .\n"
    # references_1 = ["Prospects were dim for resolution of the political crisis in Cambodia in October 1998.\nPrime Minister Hun Sen insisted that talks take place in Cambodia while opposition leaders Ranariddh and Sam Rainsy, fearing arrest at home, wanted them abroad.\nKing Sihanouk declined to chair talks in either place.\nA U.S. House resolution criticized Hun Sen's regime while the opposition tried to cut off his access to loans.\nBut in November the King announced a coalition government with Hun Sen heading the executive and Ranariddh leading the parliament.\nLeft out, Sam Rainsy sought the King's assurance of Hun Sen's promise of safety and freedom for all politicians.",
    #                 "Cambodian prime minister Hun Sen rejects demands of 2 opposition parties for talks in Beijing after failing to win a 2/3 majority in recent elections.\nSihanouk refuses to host talks in Beijing.\nOpposition parties ask the Asian Development Bank to stop loans to Hun Sen's government.\nCCP defends Hun Sen to the US Senate.\nFUNCINPEC refuses to share the presidency.\nHun Sen and Ranariddh eventually form a coalition at summit convened by Sihanouk.\nHun Sen remains prime minister, Ranariddh is president of the national assembly, and a new senate will be formed.\nOpposition leader Rainsy left out.\nHe seeks strong assurance of safety should he return to Cambodia.\n",
    #                 ]
    hypothesis_1 = result
    references_1 = reference

    all_hypothesis = [hypothesis_1]
    all_references = [references_1]

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 51.92	R: 51.92	F1: 51.92
	rouge-2:	P: 23.53	R: 23.53	F1: 23.53
	rouge-3:	P: 20.00	R: 20.00	F1: 20.00
	rouge-4:	P: 18.37	R: 18.37	F1: 18.37
	rouge-l:	P: 54.32	R: 54.32	F1: 54.32
	rouge-w:	P: 36.64	R: 21.86	F1: 27.38

Evaluation with Best
	rouge-1:	P: 51.92	R: 51.92	F1: 51.92
	rouge-2:	P: 23.53	R: 23.53	F1: 23.53
	rouge-3:	P: 20.00	R: 20.00	F1: 20.00
	rouge-4:	P: 18.37	R: 18.37	F1: 18.37
	rouge-l:	P: 54.32	R: 54.32	F1: 54.32
	rouge-w:	P: 36.64	R: 21.86	F1: 27.38

Evaluation with Individual
	Hypothesis #0 & Reference #0: 
		rouge-1:	P: 51.92	R: 51.92	F1: 51.92

	Hypothesis #0 & Reference #0: 
		rouge-2:	P: 23.53	R: 23.53	F1: 23.53

	Hypothesis #0 & Reference #0: 
		rouge-3:	P: 20.00	R: 20.00	F1: 20.00

	Hypothesis #0 & Reference #0: 
		rouge-4:	P: 18.37	R: 18.37	F1: 18.37

	Hypothesis #0 & Reference #0: 
		rouge-l:	P: 54.32	R: 54.32	F1: 54.32

	Hypothesis #0 & Reference #0: 
		rouge-w:	P: 36.64	R: 21.86	F1: 27.38


